### Notebook to parse scraped html to produce cleaned text of FCA decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International [(CC BY-NC 4.0)](https://creativecommons.org/licenses/by-nc/4.0/)

Dataset & Code to be cited as: 

    Sean Rehaag, "Federal Court of Appeal Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/fca/>.

Notes:

(1) Data Source: [Federal Court of Appeal](https://www.fca-caf.gc.ca). 

(2) Unofficial Data: The data are unofficial reproductions of materials on the Federal Court of Appeal website. Links to official versions are included in the dataset.

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Federal Court of Appeal

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, see the Federal Court of Appeal website's [Terms of Use](https://www.fca-caf.gc.ca/fca-caf_eng/important_eng.html).

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information. 

(6) Limitation: Only includes cases with neutral citation, which began to be used in 2001

(7) Delay: Decisions may take many months to be translated (sometimes over a year). As a result, in the most recent years, decisions may only be available in one language.



In [2]:
# import libraries
from bs4 import BeautifulSoup
import pandas as pd
import re
import pathlib
import json
import random

# set up progress bar
from tqdm import tqdm
tqdm.pandas()

# set paths
in_path = pathlib.Path('d:/scraping/DATA/DECISIONS/FCA/BULK/HTML/')
out_path_raw = pathlib.Path('DATA/fca_raw.jsonl')
out_path_parsed = pathlib.Path('DATA/fca_cases.jsonl')
out_path_parquet = pathlib.Path('DATA/fca_cases.parquet')
out_path_yearly = pathlib.Path('DATA/YEARLY/')

# set years sought
start_year = 2001 
end_year = 2022

# set language sought
language_sought = None  #set to 'en' if english only, set to 'fr' if French only, set to None if all languages


### Load Raw Data

In [3]:
# get list of files (including subdirectories) using pathlib
files = list(in_path.glob('**/*.json'))
print(len(files))

# Load data from files
results = []
for file in tqdm(files):
    with open(file) as f:
        data = json.load(f)
        results.append(data)

# convert list of dictionaries to dataframe
df = pd.DataFrame(results)




16800


100%|██████████| 16800/16800 [00:11<00:00, 1431.83it/s]


In [4]:
# export raw df to jsonl
df.to_json(out_path_raw, orient='records', lines=True)


### Parse Data

In [5]:
# clean dataframe

#remove ?iframe=true from scraped_link
df['scraped_link'] = df['scraped_link'].str.replace('\?iframe=true', '', regex=True)

# remove T and everything after from scraped_timestamp & case_decision_date
df['scraped_timestamp'] = df['scraped_timestamp'].str.replace('T.*', '', regex=True)
df['case_decision_date'] = df['case_decision_date'].str.replace('T.*', '', regex=True)

# remove scraped_status_code, referrer_main_source, referrer_sub_source, referrer_file, referrer_timestamp
df = df.drop(columns=['scraped_status_code', 'referrer_main_source', 'referrer_sub_source', 'referrer_file', 'referrer_timestamp'])

# convert case_year to int and filter for years sought
df['case_year'] = df['case_year'].astype(int)
df = df[df.case_year >= start_year]
df = df[df.case_year <= end_year]

# filter for langauge if desired
if language_sought:
    df = df[df.case_language == language_sought]

# remove cases where no citation (typically orders or errors)
# remove df where citation includes '=' or '-'
df = df[df.case_citation.str.contains('=') == False]
df = df[df.case_citation.str.contains('-') == False]

# if citation2 = citation 1, make citation2 = ''
df['case_citation2'] = df['case_citation2'].where(df['case_citation2'] != df['case_citation'], '')

# change 'scraped_link' column to 'source_url;
df = df.rename(columns={'scraped_link': 'source_url'})

# remove 'case_' from all column names
df.columns = df.columns.str.replace('case_', '')


In [6]:
# Extract text of cases from html
def get_text(html):

    # if html is None, return None
    if html is None:
        return None
    
    # extract text from class 'entry-content' in page1 html using beautiful soup
    soup = BeautifulSoup(html, 'html.parser')

    # convert <br> to new line to preserve paragraphs
    for br in soup.find_all('br'):
        br.replace_with('\n')

    # Insert newline characters after each <p> tag to preserve paragraphs
    for p in soup.find_all('p'):
        p.insert_after('\n')

    return soup.text

df['text'] = df.html.progress_apply(get_text)



100%|██████████| 13890/13890 [06:29<00:00, 35.64it/s]


In [7]:
# Clean text of cases
def clean_text(text):

    # remove \xa0
    text = text.replace('\xa0', ' ')

    # Remove multiple whitespaces and preserve paragraphs
    text = '\n'.join([re.sub(r'\s+', ' ', line.strip()) for line in text.split('\n')])
    
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)

    # Convert multiple newlines to single newlines
    text = re.sub(r'\n+', '\n', text)

    # Remove 'You are being directed to the most recent version...
    if 'You are being directed to the most recent version of the statute which may not be' in text:
        text = text.split('You are being directed to the most recent version of the statute which may not be')[0]

    # Remove 'Vous allez être redirigé vers la version...'
    if '\nVous allez être redirigé vers la version' in text:
        text = text.split('\nVous allez être redirigé vers la version')[0]

    # if '\nDecision Information\n' in text, remove everything before it
    if '\nDecision Information\n' in text:
        text = text.split('\nDecision Information\n')[1]

    # if '\nInformations sur la décision\n' in text, remove everything before it
    if '\nInformations sur la décision\n' in text:
        text = text.split('\nInformations sur la décision\n')[1]

    # Remove all strings '\n[Page #]\n' (with # being a number of up to 4 digits 
    text = re.sub(r'\n\[Page \d{1,3}\]\n', ' ', text)
    
    return text

df['unofficial_text'] = df.text.progress_apply(clean_text)


100%|██████████| 13890/13890 [00:25<00:00, 540.47it/s]


In [8]:
# drop unneeded columns
df = df.drop(columns=['html'])
df = df.drop(columns=['text'])

In [9]:
df

,citation,citation2,year,name,language,decision_date,source_url,scraped_timestamp,unofficial_text
1370,2001 FCA 1,,2001,Bouvidard Ltée. c. Commission de l'assurance e...,en,2001-02-01,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Bouvidard Ltée. c. Commission de l'assurance e...
1371,2001 FCA 10,,2001,Mclean v. Canada (Minister of Citizenship and ...,en,2001-02-08,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Mclean v. Canada (Minister of Citizenship and ...
1372,2001 FCA 100,,2001,Mai v. Canada (Minister of Citizenship and Imm...,en,2001-04-02,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Mai v. Canada (Minister of Citizenship and Imm...
1373,2001 FCA 101,,2001,Mitchell Verification Services Group Inc. v. C...,en,2001-04-03,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Mitchell Verification Services Group Inc. v. C...
1374,2001 FCA 102,,2001,Pontbriand v. Canada (Attorney General),en,2001-04-03,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2022-08-29,Pontbriand v. Canada (Attorney General)\nCourt...
...,...,...,...,...,...,...,...,...,...
16780,2022 CAF 93,,2022,AllStaff Inc. c. Canada (Procureur Général),fr,2022-05-30,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-04-13,AllStaff Inc. c. Canada (Procureur général)\nB...
16781,2022 CAF 96,,2022,"CSX Transportation, Inc. c. ABB Inc.",fr,2022-05-31,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-04-13,"CSX Transportation, Inc. c. ABB Inc.\nBase de ..."
16782,2022 CAF 97,,2022,Thomson c. Canada (Procureur général),fr,2022-06-01,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-04-13,Thomson c. Canada (Procureur général)\nBase de...
16783,2022 CAF 98,,2022,Brown c. Ontario (Tribunal de l'aide sociale),fr,2022-06-01,https://decisions.fca-caf.gc.ca/fca-caf/decisi...,2023-04-13,Brown c. Ontario (Tribunal de l'aide sociale)\...


### Export data

In [10]:
# export cleaned df to jsonl
df.to_json(out_path_parsed, orient='records', lines=True)

In [11]:
# export cleaned df to parquet
df.to_parquet(out_path_parquet)

In [12]:
# export cleaned df to yearly json files
for year in tqdm(range(start_year, end_year+1)):
    df[df.year == year].to_json(out_path_yearly / f'{year}.json', orient='records', indent=4)

100%|██████████| 22/22 [00:00<00:00, 26.20it/s]


In [ ]:
df